In [141]:
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import random
from sklearn import tree

data = pd.read_csv("wine_dataset.csv") 

features = data.loc[:,: "alcohol"] 
labels = data["type"]

seed = 1789
X_train_val, X_val_test, y_train_val, y_val_test = model_selection.train_test_split(features, labels, test_size=0.3, random_state=seed)
X_val, X_test, y_val, y_test = model_selection.train_test_split(X_val_test, y_val_test, test_size=0.5)
X_train, X_prune, y_train, y_prune = model_selection.train_test_split(X_train_val, y_train_val, test_size=0.20)

class Node:
    def __init__(self, feature=None, limit=None, left=None, right=None, ig=None, value=None, parent=None, majority=None, visited=False):
        self.feature = feature
        self.limit = limit
        self.left= left
        self.right = right
        self.ig = ig
        self.value = value
        self.parent = parent
        self.majority = majority # most 1 or 0
        self.visited = visited
        
    def is_leaf(self):
        return self.left is None and self.right is None


def same_label(labels):
    return labels.iloc[0]


def entropy_calculator(labels):
    tot = len(labels)
    if not tot == 0: 
        a = labels.eq(1).sum() / tot 
        b = labels.eq(0).sum() / tot 
        h = -1 * ((a * np.log2(a)) + (b * np.log2(b)))
    else:
        h = 0
    return h


def find_limit(feature):
    return feature.mean()


def conditional_calculator(features, labels, impurity):
    feature_values = []

    for feature in features:
        limit = find_limit(features[feature])
        over = labels[features[feature] >= limit]
        under = labels[features[feature] < limit]

        p_over = len(over) / len(labels) 
        p_under = len(under) / len(labels)
        
        values_over = 0
        values_under = 0
        
        if len(over.unique()) == 1:
            pass
        elif len(under.unique()) == 1:
            pass
        else:
            if impurity == "entropy":
                values_over = entropy_calculator(over) 
                values_under = entropy_calculator(under)
            elif impurity == "gini":
                values_over = gini_calculator(over) 
                values_under = gini_calculator(under)

        conditional_value = (p_over * values_over) + (p_under * values_under)
        feature_values.append([feature, conditional_value])

    return feature_values


def gini_calculator(labels):    
    tot = len(labels)
    if not tot == 0:    
        a = (labels.eq(1).sum())/len(labels)
        b = (labels.eq(0).sum())/len(labels)
        h = 1 - (a**2 + b**2) 
    else:
        h = 0
    return h


def highest_information_gain(feature_value, overall_value):
    highest_IG = ["0", 0]
    for feature, value in feature_value:
        ig = overall_value - value 
        if ig >= highest_IG[1]:
            highest_IG = [feature, ig]
    highest_feature = highest_IG[0]
    return highest_feature
    

def split_dataset(highest_IG, limit, new_features, new_labels):
    condition_X1 = new_features[highest_IG] <= limit
    condition_X2 = new_features[highest_IG] > limit
    
    X1 = new_features[condition_X1] 
    X2 = new_features[condition_X2] 

    y1 = new_labels[condition_X1] 
    y2 = new_labels[condition_X2] 

    return X1, X2, y1, y2


def impurity_type(impurity_measure, X, y):
    if impurity_measure == 'entropy':
        entropy = entropy_calculator(y) 
        conditional_entropies = conditional_calculator(X, y, 'entropy') 
        highest_IG = highest_information_gain(conditional_entropies, entropy) 
        limit = find_limit(X[highest_IG])
        X1, X2, y1, y2 = split_dataset(highest_IG, limit, X, y) 
        majority = y.mode()[0]
        return X1, X2, y1, y2, highest_IG, limit, majority
    
    elif impurity_measure == 'gini':
        overall_gini = gini_calculator(y)
        gini_index = conditional_calculator(X, y, 'gini')        
        highest_IG = highest_information_gain(gini_index, overall_gini)
        limit = find_limit(X[highest_IG])
        X1, X2, y1, y2 = split_dataset(highest_IG, limit, X, y)
        majority = y.mode()[0]
        return X1, X2, y1, y2, highest_IG, limit, majority
        
    else:
        print("Please choose a valid impurity measure")
    
    
def check_features(features):
    for feature in features:
        limit = find_limit(features[feature])
        over = []
        under = []
        for value in features[feature]:
            if value >= limit:
                over.append(1)
            else:
                under.append(0)
        if len(over) == 0 or len(under) == 0:
            return True
        else: 
            return False


            
# 1) learn()

def learn(X, y, impurity_measure): 
    if len(y.unique()) == 1:
        leaf = same_label(y) 
        return Node(value=leaf)
    
    elif check_features(X): 
        leaf = y.mode()[0]
        return Node(value = leaf)
    else: 
        X1, X2, y1, y2, highest_IG, limit, majority = impurity_type(impurity_measure, X, y)
        learn_node_left = learn(X1,y1, impurity_measure)        
        learn_node_right = learn(X2,y2, impurity_measure)
        return Node(feature = highest_IG, limit = limit, left = learn_node_left, right = learn_node_right, majority = majority )

    
# 2) Predict()
def _predict(x, node):
    if node.is_leaf():
        #y_pred.append(node.value)
        return node.value
    else:
        if x[node.feature] < node.limit:
            return _predict(x, node.left)
    return _predict(x, node.right)


def predict(X, root_node): 
        return [_predict(x, root_node)for index, x in X.iterrows()] 
    

# Task 1.1 prediction with entropy:

root_node_entropy = learn(X_train, y_train, impurity_measure='entropy')

    
# Task 1.2 prediction with gini index:

root_node_gini = learn(X_train, y_train, impurity_measure='gini')


In [142]:
# Task 1.3 Add reduced-error pruning

# 1) Full decision tree from training data:
def print_decision_tree(node, indent=""):
    print(Node(node))
    if node.is_leaf():
        print(indent + "Leaf Node: Value =", node.value)
    else:
        print(indent + "Split Node: Feature =", node.feature)
        print(indent + "  Left Branch:")
        print_decision_tree(node.left, indent + "   ")
        print(indent + "  Right Branch:")
        print_decision_tree(node.right, indent + "   ")

# Accuracy before pruning:

accuracy_model = {}

def accuracy(X, y_pred, root_node, measure):
    y_pred_ = predict(X_val, root_node_gini)
    accuracy = accuracy_score(y_val, y_pred)
    accuracy_model[measure] = accuracy
    print("Accuracy for",measure,"model:", accuracy)

    # Accuracy for model with entropy

y_pred = predict(X_val, root_node_entropy)
accuracy(X_val, y_pred, root_node_entropy, 'entropy')

    # Accuracy for model with gini

y_pred = predict(X_val, root_node_gini)
accuracy(X_val, y_pred, root_node_gini, 'gini')

def highest_accuracy(accuracy_model):
    best_accuracy = 0
    for measure in accuracy_model:
        if accuracy_model[measure] > best_accuracy:
            best_accuracy = accuracy_model[measure]
    return best_accuracy


# 2) Replace subtree T of T*:

def prune(node, parent=None): 
    acc = highest_accuracy(accuracy_model)
    node.visited = True
    if parent is None:
        pass
    if node.is_leaf(): 
        
        left = parent.left
        right = parent.right
        
        parent.left = None 
        parent.right = None
        parent.value = parent.majority

        y_pred = predict(X_prune, node)
            
        accuracy = accuracy_score(y_prune, y_pred) 
        
        if accuracy >= acc:
            acc = accuracy
        else:
            parent.left = left
            parent.right = right
            parent.value = None 
    else: 
        prune(node.left, node) 
        prune(node.right, node)
    return acc    

# 1.4 Evaluate your algorithm 

accuracy_entropy_prune = prune(root_node_entropy)
print('Accuracy after pruning entropy:',accuracy_entropy_prune)
accuracy_gini_prune = prune(root_node_gini)
print('Accuracy after pruning entropy and gini:',accuracy_gini_prune)

# The accuracy of entropy is highest before and after pruning, our best model is entropy without pruning

# Test the best model

y_pred = predict(X_test, root_node_entropy)
accuracy(X_test, y_test, root_node_entropy, 'entropy')

# Test is low, and therefore the model is not as good as hoped

Accuracy for entropy model: 0.86875
Accuracy for gini model: 0.86875
Accuracy after pruning entropy: 0.86875
Accuracy after pruning entropy and gini: 0.86875
Accuracy for entropy model: 0.5708333333333333


In [143]:
# Task 1.5 Compare to an existing implementation 

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

# Accuracy for model from Sklearn

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Sklearn:", accuracy)


Accuracy Sklearn: 0.8875


In [145]:
# Conlusion: sklearn is the best